In [49]:
# do not change the code in the block below
# __________start of block__________
import string
import os
from random import sample

import numpy as np
import torch, torch.nn as nn
import torch.nn.functional as F

from IPython.display import clear_output

import matplotlib.pyplot as plt
%matplotlib inline
# __________end of block__________

In [2]:
# do not change the code in the block below
# __________start of block__________
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print("{} device is available".format(device))
# __________end of block__________

cuda device is available


#### 1. Загрузка данных.

In [3]:
# do not change the code in the block below
# __________start of block__________
!wget https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/onegin.txt

with open('onegin.txt', 'r') as iofile:
    text = iofile.readlines()

text = "".join([x.replace('\t\t', '').lower() for x in text])
# __________end of block__________

--2025-05-25 14:41:01--  https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/onegin.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 262521 (256K) [text/plain]
Saving to: ‘onegin.txt.9’

onegin.txt.9        100%[===================>] 256.37K  --.-KB/s    in 0.09s   

2025-05-25 14:41:01 (2.92 MB/s) - ‘onegin.txt.9’ saved [262521/262521]



#### 2. Построение словаря и предобработка текста
В данном задании требуется построить языковую модель на уровне символов. Приведем весь текст к нижнему регистру и построим словарь из всех символов в доступном корпусе текстов. Также добавим токен `<sos>`.

In [4]:
# do not change the code in the block below
# __________start of block__________
tokens = sorted(set(text.lower())) + ["<sos>"]
num_tokens = len(tokens)

assert num_tokens == 84, "Check the tokenization process"

token_to_idx = {x: idx for idx, x in enumerate(tokens)}
idx_to_token = {idx: x for idx, x in enumerate(tokens)}

assert len(tokens) == len(token_to_idx), "Mapping should be unique"

print("Seems fine!")


text_encoded = [token_to_idx[x] for x in text]
# __________end of block__________

Seems fine!


__Ваша задача__: обучить классическую рекуррентную нейронную сеть (Vanilla RNN) предсказывать следующий символ на полученном корпусе текстов и сгенерировать последовательность длины 100 для фиксированной начальной фразы.

Вы можете воспользоваться кодом с занятие №6 или же обратиться к следующим ссылкам:
* Замечательная статья за авторством Andrej Karpathy об использовании RNN: [link](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)
* Пример char-rnn от Andrej Karpathy: [github repo](https://github.com/karpathy/char-rnn)
* Замечательный пример генерации поэзии Шекспира: [github repo](https://github.com/spro/practical-pytorch/blob/master/char-rnn-generation/char-rnn-generation.ipynb)

Данное задание является достаточно творческим. Не страшно, если поначалу оно вызывает затруднения. Последняя ссылка в списке выше может быть особенно полезна в данном случае.

Далее для вашего удобства реализована функция, которая генерирует случайный батч размера `batch_size` из строк длиной `seq_length`. Вы можете использовать его при обучении модели.

In [5]:
# do not change the code in the block below
# __________start of block__________
batch_size = 256
seq_length = 100
start_column = np.zeros((batch_size, 1), dtype=int) + token_to_idx["<sos>"]


def generate_chunk():
    global text_encoded, start_column, batch_size, seq_length

    start_index = np.random.randint(0, len(text_encoded) - batch_size * seq_length - 1)
    data = np.array(
        text_encoded[start_index : start_index + batch_size * seq_length]
    ).reshape((batch_size, -1))
    yield np.hstack((start_column, data))


# __________end of block__________

Пример батча:

In [6]:
next(generate_chunk())

array([[83, 50, 50, ..., 59, 54, 13],
       [83,  0, 47, ..., 53, 63, 62],
       [83, 76,  1, ...,  0, 53,  1],
       ...,
       [83,  1, 57, ..., 56, 73, 48],
       [83, 59, 54, ..., 49, 50,  1],
       [83, 51,  1, ..., 14, 44,  0]])

Далее вам предстоит написать код для обучения модели и генерации текста.

In [17]:
class ImprovedCharRNN(nn.Module):
    def __init__(self, num_tokens, emb_size=256, hidden_size=512, n_layers=3, dropout=0.3):
        super().__init__()
        self.emb = nn.Embedding(num_tokens, emb_size)
        self.lstm = nn.LSTM(emb_size, hidden_size, n_layers, 
                           dropout=dropout if n_layers > 1 else 0, 
                           batch_first=True)
        self.fc = nn.Linear(hidden_size, num_tokens)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, h_prev=None):
        x = self.emb(x)
        out, h = self.lstm(x, h_prev)
        out = self.dropout(out)
        logits = self.fc(out)
        return logits, h

In [18]:
model = ImprovedCharRNN(num_tokens).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5, factor=0.5)
criterion = nn.CrossEntropyLoss()

In [20]:
best_loss = float('inf')
patience = 10
trigger_times = 0

for epoch in range(150):
    model.train()
    total_loss = 0
    
    for _ in range(100):
        batch = next(generate_chunk())
        inputs = torch.tensor(batch[:, :-1], dtype=torch.long).to(device)
        targets = torch.tensor(batch[:, 1:], dtype=torch.long).to(device)
        
        optimizer.zero_grad()
        logits, _ = model(inputs)
        loss = criterion(logits.permute(0,2,1), targets)
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), 5.0)  # Gradient clipping
        optimizer.step()
        
        total_loss += loss.item()
    
    avg_loss = total_loss / 100
    scheduler.step(avg_loss)
    
    # Early stopping
    if avg_loss < best_loss:
        best_loss = avg_loss
        trigger_times = 0
        torch.save(model.state_dict(), 'best_model.pth')
    else:
        trigger_times += 1
        if trigger_times >= patience:
            print(f"Early stopping at epoch {epoch+1}")
            break
    
    print(f"Epoch {epoch+1}, Loss: {avg_loss:.4f}, LR: {optimizer.param_groups[0]['lr']:.2e}")

Epoch 1, Loss: 3.1491, LR: 5.00e-04
Epoch 2, Loss: 2.5758, LR: 5.00e-04
Epoch 3, Loss: 2.3736, LR: 5.00e-04
Epoch 4, Loss: 2.2311, LR: 5.00e-04
Epoch 5, Loss: 2.1175, LR: 5.00e-04
Epoch 6, Loss: 2.0091, LR: 5.00e-04
Epoch 7, Loss: 1.9122, LR: 5.00e-04
Epoch 8, Loss: 1.8140, LR: 5.00e-04
Epoch 9, Loss: 1.7349, LR: 5.00e-04
Epoch 10, Loss: 1.6427, LR: 5.00e-04
Epoch 11, Loss: 1.5617, LR: 5.00e-04
Epoch 12, Loss: 1.4908, LR: 5.00e-04
Epoch 13, Loss: 1.3921, LR: 5.00e-04
Epoch 14, Loss: 1.3160, LR: 5.00e-04
Epoch 15, Loss: 1.2561, LR: 5.00e-04
Epoch 16, Loss: 1.1734, LR: 5.00e-04
Epoch 17, Loss: 1.0918, LR: 5.00e-04
Epoch 18, Loss: 1.0436, LR: 5.00e-04
Epoch 19, Loss: 0.9699, LR: 5.00e-04
Epoch 20, Loss: 0.9039, LR: 5.00e-04
Epoch 21, Loss: 0.8560, LR: 5.00e-04
Epoch 22, Loss: 0.8012, LR: 5.00e-04
Epoch 23, Loss: 0.7682, LR: 5.00e-04
Epoch 24, Loss: 0.7019, LR: 5.00e-04
Epoch 25, Loss: 0.6329, LR: 5.00e-04
Epoch 26, Loss: 0.5962, LR: 5.00e-04
Epoch 27, Loss: 0.5626, LR: 5.00e-04
Epoch 28, 

В качестве иллюстрации ниже доступен график значений функции потерь, построенный в ходе обучения авторской сети (сам код для ее обучения вам и предстоит написать).

Шаблон функции `generate_sample` также доступен ниже. Вы можете как дозаполнить его, так и написать свою собственную функцию с нуля. Не забывайте, что все примеры в обучающей выборке начинались с токена `<sos>`.

In [ ]:
def generate_sample(
    char_rnn, seed_phrase=None, max_length=200, temperature=1.0, device=device
):
    """
    The function generates text given a phrase of length at least SEQ_LENGTH.
    :param seed_phrase: prefix characters. The RNN is asked to continue the phrase
    :param max_length: maximum output length, including seed_phrase
    :param temperature: coefficient for sampling.  higher temperature produces more chaotic outputs,
                        smaller temperature converges to the single most likely output
    """

    if seed_phrase is not None:
        x_sequence = [token_to_idx["<sos>"]] + [
            token_to_idx[token] for token in seed_phrase
        ]
    else:
        x_sequence = [token_to_idx["<sos>"]]

    x_sequence = torch.tensor([x_sequence], dtype=torch.int64).to(device)

    # feed the seed phrase, if any

    # your code here

    return "".join([tokens[ix] for ix in x_sequence.cpu().data.numpy()[0]])

In [28]:
def generate_sample(
    model, seed_phrase, max_length=500, temperature=0.7, top_k=20
):
    model.eval()
    with torch.no_grad():
        x_sequence = [token_to_idx["<sos>"]] + [token_to_idx[c] for c in seed_phrase]
        input_tensor = torch.tensor([x_sequence], dtype=torch.long).to(device)
        hidden = None
        
        for _ in range(max_length - len(x_sequence)):
            logits, hidden = model(input_tensor[:, -1:], hidden)
            logits = logits[:, -1] / temperature
            
            # Top-K filtering
            top_k_logits, top_k_indices = logits.topk(top_k, dim=-1)
            probs = F.softmax(top_k_logits, dim=-1)
            
            next_idx = torch.multinomial(probs, 1)
            next_idx = top_k_indices.gather(-1, next_idx)
            
            input_tensor = torch.cat([input_tensor, next_idx], dim=1)
            
        result = input_tensor.cpu().numpy()[0]
        return ''.join([idx_to_token[idx] for idx in result]).replace('<sos>', '')

Пример текста сгенерированного обученной моделью доступен ниже. Не страшно, что в тексте много несуществующих слов. Используемая модель очень проста: это простая классическая RNN.

In [29]:
print(
    generate_sample(
        model, " мой дядя самых честных правил", max_length=500, temperature=0.8
    )
)

 мой дядя самых честных правильенные страны,
где долго в лоно тишины
лились его живые слезы;
он пел поблеклый жизни цвет
без малого в осьмнадцать лет.



xi

в пустыне, где один евгений
мог оценить его дары,
господ соседственных селений
ему не нравились пиры;
бежал он их беседы шумной,
их разговор благоразумный
о сенокосе, о вине,
о псарне, о своей родне,
конечно, не блистал ни чувством,
ни поэтическим огнем,
ни остротою, ни умом,
ни общежития искусством;
но разговор их милых жен
гораздо меньше


### Сдача задания
Сгенерируйте десять последовательностей длиной 500, используя строку ' мой дядя самых честных правил'. Температуру для генерации выберите самостоятельно на основании визуального качества генериуремого текста. Не забудьте удалить все технические токены в случае их наличия.

Сгенерированную последовательность сохрание в переменную `generated_phrase` и сдайте сгенерированный ниже файл в контест.

In [30]:
seed_phrase = " мой дядя самых честных правил"

In [ ]:
generated_phrases = [
    generate_sample(
        model,
        ' мой дядя самых честных правил',
        max_length=501,
        temperature=0.7
    ).replace('<sos>', '')  # Удаляем служебный токен
    for _ in range(10)
]

In [45]:
generated_phrases = [
    generate_sample(model, seed_phrase, temperature=0.7, top_k=15)
    for _ in range(10)
]

In [46]:
generated_phrases.__len__()

10

In [47]:
for phrase in generated_phrases:
    print(len(phrase))

499
499
499
499
499
499
499
499
499
499


In [48]:
# do not change the code in the block below
# __________start of block__________

if "generated_phrases" not in locals():
    raise ValueError("Please, save generated phrases to `generated_phrases` variable")

for phrase in generated_phrases:

    if not isinstance(phrase, str):
        raise ValueError("The generated phrase should be a string")

    if len(phrase) != 499:
        raise ValueError("The `generated_phrase` length should be equal to 500")

    assert all(
        [x in set(tokens) for x in set(list(phrase))]
    ), "Unknown tokens detected, check your submission!"


submission_dict = {"token_to_idx": token_to_idx, "generated_phrases": generated_phrases}

np.save("submission_dict_hw09.npy", submission_dict, allow_pickle=True)
print("File saved to `submission_dict_hw09.npy`")
# __________end of block__________

File saved to `submission_dict_hw09.npy`


На этом задание завершено. Поздравляем!